In [4]:
import asyncio
import os
import time
from grainchain import Sandbox, SandboxConfig
from grainchain.core.config import ProviderConfig
from grainchain.providers.e2b import E2BProvider
from dotenv import load_dotenv

load_dotenv()
    
print("🚀 Starting E2B Demo with Custom Template")
print("=" * 50)

# Check API key
if not os.getenv("E2B_API_KEY"):
    print("❌ Error: E2B_API_KEY environment variable not set")
    print("💡 Get your API key from: https://e2b.dev/")
    raise Exception

# Configure E2B provider with our custom template
template_id = "codegen-dev-v2"
print(f"📦 Using template: {template_id}")

e2b_config = ProviderConfig(
    name="e2b",
    config={
        "api_key": os.getenv("E2B_API_KEY"),
        "template": template_id
    }
)

sandbox_config = SandboxConfig(
    timeout=600,  # 10 minutes
    working_directory="/home/user",
    environment_vars={
        "NODE_ENV": "development",
        "NODE_OPTIONS": "--max-old-space-size=8192"
    }
)

e2b_provider = E2BProvider(e2b_config)

🚀 Starting E2B Demo with Custom Template
📦 Using template: codegen-dev-v2


# Create Sandbox

In [24]:
# Step 1: Create sandbox
print("\n🔧 Step 1: Creating E2B Sandbox...")
start_time = time.time()

sandbox = Sandbox(provider=e2b_provider, config=sandbox_config)
await sandbox.create()  # This is the missing piece!

creation_time = time.time() - start_time
print(f"✅ Sandbox created in {creation_time:.2f}s")

# Step 2: Verify environment
print("\n🔍 Step 2: Verifying pre-installed environment...")
result = await sandbox.execute("""
    export NVM_DIR="$HOME/.nvm" &&
    source "$NVM_DIR/nvm.sh" &&
    echo "Node.js: $(node --version)" &&
    echo "Yarn: $(yarn --version)" &&
    echo "NPM: $(npm --version)"
""")
print("📋 Environment info:")
for line in result.stdout.strip().split("\n"):
    if line.strip():
        print(f"   {line}")


🔧 Step 1: Creating E2B Sandbox...
✅ Sandbox created in 6.55s

🔍 Step 2: Verifying pre-installed environment...
📋 Environment info:
   Node.js: v20.18.0
   Yarn: 1.22.22
   NPM: 11.4.1


# Clone Outline

In [25]:
print("\n📥 Step 3: Cloning Outline repository...")
start_time = time.time()
result = await sandbox.execute("""
    cd ~/projects &&
    git clone https://github.com/outline/outline.git &&
    cd outline &&
    echo "Repository cloned to: $(pwd)" &&
    echo "Files: $(ls -1 | wc -l) items"
""")
clone_time = time.time() - start_time
if result.success:
    print(f"✅ Clone completed in {clone_time:.2f}s")
    for line in result.stdout.strip().split("\n"):
        if line.strip():
            print(f"   {line}")
else:
    print(f"❌ Clone failed: {result.stderr}")


📥 Step 3: Cloning Outline repository...
✅ Clone completed in 13.44s
   Repository cloned to: /home/user/projects/outline
   Files: 25 items


# yarn install

In [26]:
# Step 4: Check Node version compatibility
print("\n🔧 Step 4: Checking Node.js compatibility...")
result = await sandbox.execute("""
    cd ~/projects/outline &&
    echo "Required Node version:" &&
    grep -A 3 '"engines"' package.json || echo "No engine requirements found"
""")
print("📋 Compatibility check:")
for line in result.stdout.strip().split("\n"):
    if line.strip():
        print(f"   {line}")

# Step 5: Install dependencies
print("\n📦 Step 5: Installing dependencies with yarn...")
start_time = time.time()
result = await sandbox.execute("""
    cd ~/projects/outline &&
    export NVM_DIR="$HOME/.nvm" &&
    source "$NVM_DIR/nvm.sh" &&
    yarn install --frozen-lockfile &&
    echo "Dependencies installed successfully!" &&
    echo "node_modules size: $(du -sh node_modules 2>/dev/null || echo 'N/A')" &&
    echo "Package count: $(ls node_modules 2>/dev/null | wc -l || echo '0')"
""")
install_time = time.time() - start_time
if result.success:
    print(f"✅ Installation completed in {install_time:.2f}s")
    # Show last few lines of output
    lines = result.stdout.strip().split("\n")
    for line in lines[-5:]:
        if line.strip():
            print(f"   {line}")
else:
    print(f"❌ Installation failed in {install_time:.2f}s")
    print(f"Error: {result.stderr[:200]}..." if len(result.stderr) > 200 else result.stderr)


🔧 Step 4: Checking Node.js compatibility...
📋 Compatibility check:
   Required Node version:
     "engines": {
       "node": ">= 18 <=20"
     },
     "repository": {

📦 Step 5: Installing dependencies with yarn...


# Snapshot

In [29]:
# Step 6: Create snapshot
print("\n📸 Step 6: Creating development snapshot...")
start_time = time.time()
snapshot_name = "outline_dev_ready"
result = await sandbox.execute(f"""
    cd ~/projects &&
    echo "=== E2B Outline Development Snapshot ==="  > {snapshot_name}.log &&
    echo "Created: $(date)" >> {snapshot_name}.log &&
    echo "Node.js: $(node --version)" >> {snapshot_name}.log &&
    echo "Directory: $(pwd)" >> {snapshot_name}.log &&
    tar -czf {snapshot_name}.tar.gz outline/ &&
    echo "Snapshot created: {snapshot_name}.tar.gz" &&
    echo "Size: $(du -sh {snapshot_name}.tar.gz | cut -f1)"
""")
snapshot_time = time.time() - start_time

if result.success:
    print(f"✅ Snapshot created in {snapshot_time:.2f}s")
    for line in result.stdout.strip().split("\n"):
        if line.strip():
            print(f"   {line}")
else:
    print(f"❌ Snapshot failed: {result.stderr}")


📸 Step 6: Creating development snapshot...
✅ Snapshot created in 5.88s
   Snapshot created: outline_dev_ready.tar.gz
   Size: 77M


In [2]:
    # Step 1: Create sandbox
    print("\n🔧 Step 1: Creating E2B Sandbox...")
    start_time = time.time()

    async with Sandbox(provider=e2b_provider, config=sandbox_config) as sandbox:
        creation_time = time.time() - start_time
        print(f"✅ Sandbox created in {creation_time:.2f}s")

        # Step 2: Verify environment
        print("\n🔍 Step 2: Verifying pre-installed environment...")
        result = await sandbox.execute("""
            export NVM_DIR="$HOME/.nvm" &&
            source "$NVM_DIR/nvm.sh" &&
            echo "Node.js: $(node --version)" &&
            echo "Yarn: $(yarn --version)" &&
            echo "NPM: $(npm --version)"
        """)
        print("📋 Environment info:")
        for line in result.stdout.strip().split("\n"):
            if line.strip():
                print(f"   {line}")

        # Step 3: Clone Outline
        print("\n📥 Step 3: Cloning Outline repository...")
        start_time = time.time()
        result = await sandbox.execute("""
            cd ~/projects &&
            git clone https://github.com/outline/outline.git &&
            cd outline &&
            echo "Repository cloned to: $(pwd)" &&
            echo "Files: $(ls -1 | wc -l) items"
        """)
        clone_time = time.time() - start_time
        if result.success:
            print(f"✅ Clone completed in {clone_time:.2f}s")
            for line in result.stdout.strip().split("\n"):
                if line.strip():
                    print(f"   {line}")
        else:
            print(f"❌ Clone failed: {result.stderr}")
            return

        # Step 4: Check Node version compatibility
        print("\n🔧 Step 4: Checking Node.js compatibility...")
        result = await sandbox.execute("""
            cd ~/projects/outline &&
            echo "Required Node version:" &&
            grep -A 3 '"engines"' package.json || echo "No engine requirements found"
        """)
        print("📋 Compatibility check:")
        for line in result.stdout.strip().split("\n"):
            if line.strip():
                print(f"   {line}")

        # Step 5: Install dependencies
        print("\n📦 Step 5: Installing dependencies with yarn...")
        start_time = time.time()
        result = await sandbox.execute("""
            cd ~/projects/outline &&
            export NVM_DIR="$HOME/.nvm" &&
            source "$NVM_DIR/nvm.sh" &&
            yarn install --frozen-lockfile &&
            echo "Dependencies installed successfully!" &&
            echo "node_modules size: $(du -sh node_modules 2>/dev/null || echo 'N/A')" &&
            echo "Package count: $(ls node_modules 2>/dev/null | wc -l || echo '0')"
        """)
        install_time = time.time() - start_time

        if result.success:
            print(f"✅ Installation completed in {install_time:.2f}s")
            # Show last few lines of output
            lines = result.stdout.strip().split("\n")
            for line in lines[-5:]:
                if line.strip():
                    print(f"   {line}")
        else:
            print(f"❌ Installation failed in {install_time:.2f}s")
            print(f"Error: {result.stderr[:200]}..." if len(result.stderr) > 200 else result.stderr)

        # Step 6: Create snapshot
        print("\n📸 Step 6: Creating development snapshot...")
        start_time = time.time()
        snapshot_name = "outline_dev_ready"
        result = await sandbox.execute(f"""
            cd ~/projects &&
            echo "=== E2B Outline Development Snapshot ==="  > {snapshot_name}.log &&
            echo "Created: $(date)" >> {snapshot_name}.log &&
            echo "Node.js: $(node --version)" >> {snapshot_name}.log &&
            echo "Directory: $(pwd)" >> {snapshot_name}.log &&
            tar -czf {snapshot_name}.tar.gz outline/ &&
            echo "Snapshot created: {snapshot_name}.tar.gz" &&
            echo "Size: $(du -sh {snapshot_name}.tar.gz | cut -f1)"
        """)
        snapshot_time = time.time() - start_time

        if result.success:
            print(f"✅ Snapshot created in {snapshot_time:.2f}s")
            for line in result.stdout.strip().split("\n"):
                if line.strip():
                    print(f"   {line}")
        else:
            print(f"❌ Snapshot failed: {result.stderr}")

        # Summary
        total_time = creation_time + clone_time + install_time + snapshot_time
        print("\n🎯 Demo Summary:")
        print(f"   Sandbox creation: {creation_time:.2f}s")
        print(f"   Git clone: {clone_time:.2f}s")
        print(f"   Yarn install: {install_time:.2f}s")
        print(f"   Snapshot: {snapshot_time:.2f}s")
        print(f"   Total time: {total_time:.2f}s")

        print("\n✨ E2B Demo completed successfully!")
            

NameError: name 'snapshot' is not defined